In [1]:
import examples, test, solution # Визуализация примеров, тесты и интерактив с реализацией
from IPython.display import HTML
HTML(open("style.css", "r").read())

### Введение
Изначально трапецоидную карту придумали для решения задачи локализации точки на плоскости.
Хотя сейчас для локализации применяются другие, более простые в реализации алгоритмы, трапецоидные карты могут применяться и в других задачах, например, в построении навигационных графов.

Мы рассмотрим рандомизированный инкрементальный алгоритм построения трапецоидной карты. Карта _в среднем_ использует  $O(n)$ памяти, строится за $O(n\log n)$ и отвечает на запрос локализации точки за $O(\log n)$.

### Трапецоидная карта
Изначально на плоскости имеется множество отрезков без внутренних пересечений (точка отрезка называется _внутренней_, если она не является концом отрезка, соответственно _внутреннее пересечение_ — пересечение внутренних точек двух отрезков). Это множество "условно" помещается в bounding box $R$ — прямоугольник, ограничивающий все отрезки. Также условимся, что среди всех вершин отрезков никакие две различные вершины не лежат на одной вертикальной прямой (но при этом вершины могут совпадать). В дальнейшем эти ограничения можно будет снять.

Трапецоидная карта — структура данных для локализации точки среди этих отрезков. Она получается следующим путем: из каждой вершины выпускается два вертикальных луча, вверх и вниз, до первого пересечения с другим отрезком или с $R$. В результате получаются замкнутые многоугольники — трапецоиды.

![Пример 1](images/tmap_1.jpg)
<i><center>Рисунок 1. Пример трапецоидной карты</center></i>

Рассмотрим, что из себя представляет отдельно взятый трапецоид:

> __Лемма.__ _Любой трапецоид ограничен одним или двумя вертикальными и двумя не вертикальными отрезками_

$\triangleright$
<div style="padding-left:40px">
Обозначим наш трапецоид <b>f</b>. 
Для начала покажем, что <b>f</b> выпуклый. 
Любая угловая точка у <b>f</b> по построению является либо концом отрезка, либо пересечением вертикального луча с другим отрезком или с $R$, либо это один из углов $R$. 
Угол пересечения вертикального луча с отрезком не превзойдет 180 градусов, а угол с участием $R$ равен 90 градусам. 
Следовательно, <b>f</b> выпуклый, так как вертикальные лучи устраняют все невыпуклости.

По построению у <b>f</b> должно быть не более 2 вертикальных отрезков (один из них может отсутствовать) и не менее 2 не вертикальных. Допустим, не вертикальных будет больше 2. Тогда как минимум 2 из них будут смежными с верхней (или нижней) стороны, и у них будет общая точка. Но в ней должен был быть вертикальный луч, который создал бы дополнительный трапецоид! Значит, не вертикальных отрезков ровно два, а всего у <b>f</b> либо 3, либо 4 стороны.
</div>
$\triangleleft$

Отсюда и берется название трапецоидных карт, так как трапецоид представляет из себя либо трапецию, либо треугольник.

Обозначим отрезок, лежащий сверху трапецоида, $\mathit{top}(\Delta)$ и аналогично лежащий снизу $\mathit{bottom}(\Delta)$. Рассмотрим возможные варианты расположения левого отрезка в трапецоиде:

<div><p id="my_p">__a)__ он отсутствует, вместо него точка пересечения $\mathit{top}(\Delta)$ и $\mathit{bottom}(\Delta)$;</p></div>
<div><p id="my_p">__b)__ он образован лучом, идущим вниз из левой точки $\mathit{top}(\Delta)$;</p></div>
<div><p id="my_p">__c)__ он образован лучом, идущим вверх из левой точки $\mathit{bottom}(\Delta)$;</p></div>
<div><p id="my_p">__d)__ он образован двумя лучами из правой точки отрезка, лежащего слева от $\Delta$;</p></div>
<div><p id="my_p">__e)__ это левая граница $R$.</p></div>

![Случаи расположения leftp](images/leftp_cases.jpg)
<i><center>Рисунок 2. Варианты расположения левой точки трапецоида</center></i>

В каждом случае (за исключением рис. 2e) левый отрезок определяет одна точка $p$ — вершина одного из отрезков. Обозначим ее как $\mathit{leftp}(\Delta)$ (в случае для $R$ это будет $\mathit{None}$, так как координата его левой нижней точки не известна, а bounding box $R$ мы выдумали). Аналогичным способом получим 5 возможных случаев для расположения правого отрезка, обозначив правую вершину как $\mathit{rightp}(\Delta)$. Заметим, что трапецоид однозначно задается набором $\mathit{top}(\Delta)$, $\mathit{bottom}(\Delta)$, $\mathit{leftp}(\Delta)$, $\mathit{rightp}(\Delta)$.


А сколько же всего трапецоидов мы получим?

> <b>Лемма.</b> _Трапецоидная карта, построенная на $n$ отрезках, содержит не более $6n+4$ вершин и $3n+1$ трапецоидов_

$\triangleright$
<div style="padding-left:40px">
Вершиной трапецоида может являться либо "условная" вершина $R$ (всего их четыре), либо конец отрезка ($2n$ вершин), либо конец вертикального луча, выходящего из конца отрезка ($2n \cdot 2=4n$ вершин). Итого не более $6n+4$ вершин.

Для ограничения числа трапецоидов рассмотрим точку $\mathit{leftp}(\Delta)$. Она является либо концом отрезка, либо вершиной $\mathit{R}$. Если это вершина $\mathit{R}$, то этот трапецоид ограничен левой стороной $\mathit{R}$, а такой трапецоид будет один (рис. 2e). Правый конец каждого отрезка задает не более одной $\mathit{leftp}(\Delta)$ (рис. 2d), левый конец отрезка не более двух $\mathit{leftp}(\Delta)$ (рис. 2b и рис. 2c). Однако на рис. 2a, когда $k>1$ отрезков имеют общую левую точку, $\mathit{leftp}(\Delta)$ может быть общей сразу для $k+1$ трапецоидов. В этом случае условимся, что каждый из $k$ отрезков задает $\mathit{leftp}(\Delta)$ лишь для верхнего и нижнего трапецоида, тогда каждому трапецоиду соответствует два отрезка и все хорошо. Значит, каждая точка $\mathit{leftp}(\Delta)$ задает не более трех трапецоидов, а общее количество трапецоидов не превышает $3n+1$.
</div>
$\triangleleft$

В дальнейшем нам понадобятся трапецоиды с общей вертикальной прямой, назовем их соседями. 
По нашему условию на одной вертикальной прямой лежит максимум одна вершина, значит трапецоид может иметь до двух соседних трапецоидов слева и справа (иначе бы могло быть любое число соседей). 
На рис. 2a и 2e у трапецоида левые соседи отсутствуют, на рис. 2b присутствует только нижний, на рис. 2c только верхний, а на рис. 2d сразу оба соседа. 
Также трапецоиду необходимо знать узлы, которые на него указывают (в процессе построения локализационной структуры станет ясно, зачем).

В итоге классы для трапецоидной карты выглядят _приблизительно_ так:
 <details id="custom"><summary><span>Класс, описывающий отрезок</span></summary>
```python
class Segment:

    def __init__(self, p: Point, q: Point):
        # Точки p и q упорядочены лексикографически"
        assert p.__lt__(q)
        self.p = p  # Левая точка
        self.q = q  # Правая точка
```
</details>
 <details id="custom"><summary><span>Класс, описывающий трапецоид</span></summary>
   <p>
```python
class Trapezoid:

    def __init__(self, top: Segment, bottom: Segment, leftp: Point, rightp: Point):
        # Верхний и нижний отрезки
        self.top = top
        self.bottom = bottom

        # Левая и правая точки
        self.leftp = leftp
        self.rightp = rightp

        # Соседи трапецоида
        self.leftnb = [None, None]
        self.rightnb = [None, None]

        # Ссылка на узел локализационной структуры
        self.node = None
```
</p></details>

Полная реализация этих классов находится в файле [TMapClasses.py](TMapClasses.py).

### Локализация точки на трапецоидной карте
Во время построения трапецоидной карты также строится локализационная структура, которая позволяет локализовать точку на карте. Локализационная структура представляет из себя ациклический ориентированный граф с одним корнем (практически дерево, но в узел графа может входить более одного ребра), в котором листами являются трапецоиды.

![Пример 2](images/tmap_2.jpg)
<i><center>Рисунок 3. Пример локализационной структуры и трапецоидной карты</center></i>

__Важное замечание:__ каждому трапецоиду в графе соответствует ровно один лист!

Все узлы графа делятся на 2 типа:

- $X$ соответствует вершине отрезка ($p_i$ левая вершина, а $q_i$ правая)
- $Y$ соответствует самому отрезку $s_i$

У каждого узла графа ровно 2 исходящих ребра. 
При запросе локализации точки $q$ на трапецоидной карте мы спускаемся по графу от корня к нужному трапецоиду. 
В случае узла $X$ мы сравниваем вершины лексикографически: если $q$ меньше, то идем по левому ребру, иначе по правому. 
В случае узла $Y$ мы проверяем положение $q$ относительно отрезка. 
Если $q$ выше, то идем по левому ребру, иначе по правому. 
В конце мы доходим до узла трапецоида, которому принадлежит точка $q$.

Узлы каждого типа реализованы в [TMapClasses.py](TMapClasses.py) примерно так:
 <details id="custom"><summary><span>Базовый узел</span></summary>
```python
class AbstractNode:

    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def visit(self, s):
        """Возвращает трапецоид, в котором лежит точка"""
        pass
```
</details>
 <details id="custom"><summary><span>Узел типа ___X___</span></summary>
```python
class XNode(AbstractNode):

    def __init__(self, point, left, right):
        super().__init__(left, right)
        self.point = point

    def visit(self, s):
        # Порядок обхода задается лексикографическим сравнением точек
        return self.left.visit(s) if s.p < self.point else self.right.visit(s)
```
</details>
 <details id="custom"><summary><span>Узел типа ___Y___</span></summary>
```python
class YNode(AbstractNode):

    def __init__(self, segment, left, right):
        super().__init__(left, right)
        self.segment = segment

    def visit(self, s):
        # Порядок обхода задает предикат поворота
        sign = turn(s.p, self.segment.p, self.segment.q)
        return self.left.visit(s) if sign == 1 else self.right.visit(s)
```
</details>
 <details id="custom"><summary><span>Узел трапецоида</span></summary>
```python
class TrapezoidNode(AbstractNode):

    def __init__(self, trapezoid):
        super().__init__()
        # Ссылка на трапецоид
        self.tr = trapezoid
        trapezoid.node = self
        # Ссылки на все узлы, которые указывают на трапецоид
        self.links = []

    def visit(self, point):
        return self.tr
```
</details>

Метод <code>visit</code> позволяет рекурсивно локализовать точку. 
Таким образом, для локализации точки достаточно запустить его от корня локализационной структуры.

Рассмотрим процесс локализации точки $q$ на трапецоидной карте побольше, изображенной на рис. 4.

![Большой пример](images/tmap_3.jpg)
<i><center>Рисунок 4. Локализация точки на трапецоидной карте</center></i>

In [ ]:
examples.slideshow('map', 1200) #период, мс

Возможны случаи, когда $q$ совпадает с вершиной или принадлежит отрезку. С одной стороны, нельзя не признать, что $q$ все ещё принадлежит трапецоиду (причем, сразу нескольким), и при выборе любого ребра, исходящего из текущего узла, мы придем к нужному трапецоиду.
С другой стороны, нельзя не согласиться, что вершина или отрезок более точно локализуют точку запроса, нежели трапецоид, значит мы можем выдать в качестве ответа отрезок, на котором лежит $q$ (или в случае вершины отрезок, который ей соответствует). 
Как на самом деле реализован алгоритм локализации будет сказано в конце конспекта.

### Построение поисковой структуры и трапецоидной карты
Изначально карта $J_0$ состоит из единственного трапецоида, у которого отсутствуют и соседи, и верхний/нижний отрезок, и левая/правая точка (фактически, это весь $R$). Также он лежит в корне $D$. 
 <details id="custom"><summary><span>Класс трапецоидной карты</span></summary>
```python
class TrapezoidalMap():

    def __init__(self):
        # Список всех трапецоидов на карте, изначально имеется единственный "пустой" трапецоид
        self.tr = [Trapezoid(None, None, None, None)]
        # Список вставленных отрезков
        self.segments = []
        # Корень поисковой структуры
        # Изначально в него помещается узел, соответствующий "пустому" трапецоиду
        self.root = TrapezoidNode(self.tr[0])
```
</details>

Алгоритм построения трапецоидной карты инкрементальный: в уже существующую карту по одному добавляются новые отрезки. Что необходимо сделать при добавлении очередного отрезка $s_i$:
- найти трапецоиды $\Delta_0,\Delta_1,\ldots,\Delta_k$, которые пересекает $s_i$
- удалить их из $J_{i-1}$ и заменить на новые трапецоиды, появившиеся при вставке $s_i$
- заменить листы из $D_{i-1}$, соответствующие старым трапецоидам, на новые

Поиск $\Delta_0,\Delta_1,\ldots,\Delta_k$ выполняется довольно просто. Сначала мы находим $\Delta_0$, локализуя левую точку отрезка $s_i$ в $D_{i-1}$ за $O(h)$, где $h$ — высота $D_{i-1}$. Далее легко получить $\Delta_1,\ldots,\Delta_k$, проходя вправо по соседям трапецоидов. Для проверки, верхним или нижним будет следующий трапецоид, нужно проверить поворот точки $\mathit{rightp}(\Delta_j)$ относительно прямой $s_i$. Поиск остановится, когда правая вершина $s_i$ либо окажется левее $\mathit{rightp}(\Delta_k)$, либо попадет в крайний правый трапецоид, либо конец отрезка совпадет с $\mathit{rightp}(\Delta_j)$. Таким образом мы получим $\Delta_0,\Delta_1,\ldots,\Delta_k$ за $O(h+k)$.

### Упражнение: поиск пересеченных трапецоидов
Реализуйте поиск $\Delta_0,\Delta_1,\ldots,\Delta_k$, используя предикат поворота. 
Ваша функция должна вернуть все трапецоиды (кроме первого) в том порядке, в котором их пересекает отрезок. 
Гарантируется, что исходный трапецоид содержит в себе более одной точки отрезка. 
Набор трапецоидов должен совпасть с результатом <code>solution.intersect_segment</code> из [solution.py](solution.py).

In [ ]:
def intersect_segment(segment, trapezoid):
    """Возвращает список трапецоидов, в которые попал отрезок"""
    return solution.intersect_segment(segment, trapezoid)

test.intersection_test(intersect_segment)

Далее необходимо удалить старые трапецоиды и на их место вставить новые. Сначала разберем простой случай, когда $s_i$ целиком попал в один трапецоид $\Delta$. Вместо него появятся 4 новых трапецоида. Необходимо обновить указатели у соседей $\Delta$ и новых трапецоидов, а также заменить в $D_{i-1}$ лист, соответствовавший $\Delta$, на новое поддерево высоты 3, как показано на рис. 5. Этот случай обрабатывается за $O(1)$.

![Один трапецоид](images/single_insert.jpg)
<i><center>Рисунок 5. Вставка отрезка в трапецоид</center></i>

### Упражнение: простой случай вставки
Реализуйте простой случай вставки отрезка в трапецоидную карту. Ваша функция должна работать так же, как <code>solution.insert_segment</code> из [solution.py](solution.py). Разрешено использовать функцию <code>solution.localize</code> для поиска трапецоида, в который попал отрезок. Гарантируется, что отрезок целиком попадает ровно в один трапецоид, а концы этого отрезка не лежат на уже существующих вершинах.

In [ ]:
#print(help(solution.localize))

def simple_insert(tmap, segment):
    localized_trapezoid = solution.localize(tmap, segment)
    return solution.insert_segment(tmap, segment, localized_trapezoid)
    
test.simple_insert_test(simple_insert)

Перейдем к сложному случаю, когда $s_i$ пересекает $\Delta_0,\Delta_1,\ldots,\Delta_k$. Возможны 3 варианта развития событий:
- левый конец отрезка внутри $\Delta_0$
- правый конец отрезка внутри $\Delta_k$
- отрезок полностью пересекает трапецоид

В первых двух случаях концы $s_i$ порождают новые вертикальные лучи, то есть необходимо разбить $\Delta_0$ и/или $\Delta_k$ на три трапецоида. 
Кроме того, $s_i$ пересечет некоторые другие вертикальные лучи, значит надо перестроить трапецоиды вдоль $s_i$, начиная с $\Delta_0$ или $\Delta_1$. 
При разбиении $\Delta_j$ обозначим трапецоид ниже $s_i$ как $\Delta_{down}$, а выше как $\Delta_{up}$. 
Так как $s_i$ пересечет вертикальный луч из $\mathit{rightp}(\Delta_j)$, у одного из новых трапецоидов точка $\mathit{rightp}$ окажется правее $\mathit{rightp}(\Delta_j)$, поэтому нужно определить поворот $\mathit{rightp}(\Delta_j)$ относительно $s_i$. 
Если она лежит сверху, то $\Delta_{down}$ продолжится вдоль отрезка, а $\Delta_{up}$ закончится. 
Теперь можно обновить все вершины, отрезки и соседей для $\Delta_{down}$ и $\Delta_{up}$, кроме $\mathit{rightp}(\Delta_{down})= \mathit{None}$. 
Запомним, что $\Delta_{down}$ еще не достроен, и перейдем к следующему трапецоиду. 
$\mathit{rightp(\Delta_{down})}$ будет известна, когда появится такой трапецоид $\Delta_p$, что $\mathit{rightp}(\Delta_p)$ будет ниже $s_i$, или мы дойдем до конца отрезка $s_i$. 
Если же вдоль отрезка продолжится $\Delta_{up}$, то поступаем аналогичным образом.

В $D_{i}$ листы, соответствовавшие $\Delta_0,\Delta_1,\ldots,\Delta_k$, заменяются на новые поддеревья. На рис. 6 показан пример обновления трапецоидной карты и локализационной структуры.

![Несколько трапецоидов](images/multi_insert.jpg)
<i><center>Рисунок 6. Вставка отрезка в несколько трапецоидов</center></i>

В первом и втором случае старый трапецоид заменяется на поддерево высотой 3 с узлами типа $X$ и $Y$, которые указывают на 3 новых трапецоида. Все остальные трапецоиды подпадают под третий случай, где лист заменится на поддерево высоты 2 с узлом типа $Y$, указывающим на два новых трапецоида. Структура обновляется за $O(k)$, так как для каждого из $k+1$ трапецоида выполняется $O(1)$ действий. В итоге высота дерева увеличивается не более, чем на 2. 

Теперь вернемся к большому примеру трапецоидной карты (см. рис. 4) и детально рассмотрим вставку очередного отрезка.

![Вставка в большом примере](images/map_insert.gif)
<i><center>Рисунок 7. Обновление карты после вставки отрезка</center></i>

In [ ]:
examples.slideshow('insert', 1500) #период, мс

Как видно, новый отрезок $s_{17}$ пересекает 4 трапецоида. Вершины отрезка попадают внутрь трапецоидов $\Delta_0$ и $\Delta_{14}$, поэтому в новом дереве вместо этих трапецоидов сначала вставляются $p_{17}$ и $q_{17}$ (выделены желтым), а уже затем сам отрезок (как и для двух других трапецоидов $\Delta_{5}$ и $\Delta_{10}$).

### Упражнение: вставка непересекающихся отрезков
Реализуйте алгоритм для непересекающихся отрезков.
Это следующий шаг реализации алгоритма, так что вам понадобятся функции <code>simple_insert</code> и <code>intersect_segment</code> из упражнений выше.
Разрешено использовать функцию <code>solution.localize</code> для поиска трапецоида, в который попал отрезок.
Гарантируется, что отрезки не пересекаются.

In [ ]:
def non_crossing_insert(tmap, segment):
    # localized_trapezoid = solution.localize(tmap, segment)
    return solution.insert(tmap, segment)
    
test.non_crossing_insert_test(non_crossing_insert)

### Вырожденные случаи
#### Вершины на одной вертикальной прямой

Ранее мы условились, что среди всех вершин отрезков любые две вершины не могут лежать на одной вертикальной прямой. 
Разрешим эти случаи. 
_Мысленно_ выполним так называемое преобразование сдвига (<i>англ.</i> <a href="https://en.wikipedia.org/wiki/Shear_mapping">shear mapping</a> или skew), применив преобразование для всех точек отрезков:

$\varphi : \left( \begin{array}{c} x\\ y\end{array}\right) 
\rightarrow
\left( \begin{array}{c} x + \varepsilon y\\ y\end{array}\right)$

где $\varepsilon > 0$ коэффициент сдвига. 
Горизонтальные прямые не поменяют своего положения, в то время как вертикальные перейдут в прямые с углом наклона $1/ \varepsilon$. 
При достаточно малом $\varepsilon$ никакие две точки не будут лежать на одной вертикальной прямой, а также исходный порядок точек по координате $x$ не поменяется, и все будет хорошо. 
Конечно, появятся новые вырожденные трапецоиды, которые не могли возникнуть в исходной система координат. 
Но так как количество отрезков не изменилось, время работы алгоритма останется тем же.

Заметим, что локализационная структура на _сдвинутых_ отрезках будет совпадать с локализационной структурой, построенной на исходных отрезках, если в $X$-узлах мы будем использовать <b>лексикографическое сравнение</b>.

#### Совпадающие вершины
Вершины $s_i$ (левая $p_i$ и правая $q_i$) могут совпадать с вершинами ранее вставленных отрезков. 
Это усложнит реализацию алгоритма по нескольким причинам.
Во-первых, при проходе по $\Delta_0,\Delta_1,\ldots,\Delta_k$ нужно вовремя понять, что $q_i$ попала в $\mathit{rightp}(\Delta_k)$. 
Также $p_i$ могла попасть в $\mathit{leftp}(\Delta_0)$. 
Или сразу обе вершины...

Во-вторых, усложнится алгоритм локализации. 
Рассмотрим случай, когда $p_i$ совпадает с вершиной другого вставленного отрезка. 
При локализации в узлах типа $X$ эти вершины будут равны. 
Но мы локализуемся для поиска самого левого трапецоида, в котором лежит отрезок. 
Значит трапецоиды, находящиеся слева от вертикальной прямой, проходящей через $p_i$, нам не интересны. 
Поэтому в узлах $X$ будем идти влево, если $p_i$ <b>строго</b> меньше вершины узла $X$, а иначе вправо.

В узлах типа $Y$ поворот для $p_i$ будет равен $0$. 
Чтобы понять, выше или ниже находится новый отрезок, нам понадобится $q_i$. 
Аналогично проверим ее поворот относительно отрезка в узле $Y$. 
Если $q_i$ выше, то идем по левому ребру, иначе по правому. 
Если и этот поворот равен $0$, то оба отрезка имеют внутреннее пересечение, чего не должно быть по условию.

Таким образом, для алгоритма локализации понадобится весь отрезок. 
При этом время работы локализации осталось тем же, так как мы лишь увеличили константу за счет дополнительного поворота, а глубина локализационной структуры не изменилась.

В итоге локализация и вставка нового отрезка $s_i$ займут $O(h+k)$ времени. На выходе мы получим корректную трапецоидную карту $J$ и поисковую структуру $D$, так как на каждом шаге добавление нового отрезка было корректным.

### Асимптотика и память
Порядок добавления отрезков очень важен, так как при добавлении нового отрезка высота локализационной структуры $D$ может увеличится до 3, а в худшем случае высота дерева может составить $3n$.
Несложно придумать последовательность отрезков, демонстрирующую этот случай.
Тогда алгоритм будет строить дерево за $O(n^2)$, а локализация точки будет выполняться за $O(n)$.
Для сглаживания этой неприятности будем добавлять отрезки в случайном порядке, что дает более приемлемую временную оценку.

Зафиксируем множество $S$ из $n$ отрезков и точку запроса $q$.
Всего возможно $n!$ перестановок отрезков, а значит $n!$ различных структур $D$.
В этом случае мы можем оценить ожидаемое значение высоты $D$.
На $i$-ой итерации алгоритма вставили отрезок $s_i$.
Попробуем локализовать точку $q$.
Добавим немножко теорвера: обозначим количество узлов на пути локализации $q$, созданных на $i$-ой итерации, за $x_i$.
Это случайная величина.
Найдем матожидание длины пути:

$E\left[\sum_{i=1}^{n}{x_i}\right]=\sum_{i=1}^n{E\left[x_i\right]}$

Также мы знаем, что $x_i\leq3$. Обозначим $p_i$ как вероятность встретить на пути локализации $q$ узел, созданный на $i$-ой итерации. Ясно, что $E\left[x_i\right]\leq3p_i$. $p_i\neq0$ только тогда, когда $q \in \Delta_{i-1}$, но на шаге $i$ трапецоид $\Delta_{i-1}$ был удален, а точка $q$ перешла в трапецоид $\Delta_i$.
Применим так называемый "backwards-analysis": на $i$ шаге удалим случайный отрезок $s'_i$ и оценим вероятность исчезновения трапецоида $\Delta_i$. Это произойдет в 4 случаях:
- $\mathit{top}(\Delta_i)=s'_i$
- $\mathit{bottom}(\Delta_i)=s'_i$
- $\mathit{leftp}(\Delta_i)$ является концом отрезка $s'_i$
- $\mathit{rightp}(\Delta_i)$ является концом отрезка $s'_i$

Так как отрезки вставлялись в случайном порядке, то для каждого случая вероятность того, что $s'_i=s_i$ равна $1/i$, а в сумме она не превосходит $4/i$. Таким образом $\sum_{i=1}^n{E\left[x_i\right]} \leq \sum_{i=1}^n{3p_i} \leq \sum_{i=1}^n{12/i} = 12\sum_{i=1}^n{1/i} = 12H_n$, где $H_n$ — гармонический ряд, который равен $12\ln(n) = O(\log n )$. Значит, ожидаемое время локализации составит $O(\log n )$.

Теперь вернемся к размеру $D$. В худшем случае на каждой итерации алгоритма новый отрезок будет пересекать все трапецоиды, и тогда размер структуры составит $O(n^2)$. Найдем ожидаемый объем памяти. Мы знаем, что в конце алгоритма у нас будет $O(n)$ листов в $D$, тогда размер $D$ составит $O(n) + \sum_{i=1}^{n}{E\left[x_i-1\right]} = O(n) + \sum_{i=1}^{n}{E\left[x_i\right]}$, где $x_i$ — количество трапецоидов, созданных на $i$-ой итерации. Надо ограничить $E\left[x_i\right]$.

Снова применим backwards-analysis. Зафиксируем набор отрезков $S_i$ и введем новую функцию:

$\delta(\Delta, s) = \begin{cases}
1, &\text{если }\Delta\text{ исчезнет при удалении }s_i\text{;}\\
0, &\text{иначе.}
\end{cases}$

$\Delta$ может исчезнуть при удалении $\mathit{top}(\Delta)$, $\mathit{bottom}(\Delta)$, точки $\mathit{leftp}(\Delta)$ или $\mathit{rightp}(\Delta)$ (если они присутствуют у $\Delta$). Значит, на $\Delta$ влияет не более 4 отрезков, тогда $\sum_{s \in S_i}{\sum_{\Delta \in J_i} {\delta(\Delta, s)}} \leq 4 \left|J_i\right| = O(i)$. С учетом того, что вероятность удаления отрезка равна $1/i$, найдем матожидание: $E\left[x_i\right] = 1/i \sum_{s \in S_i}{\sum_{\Delta \in J_i}{\delta(\Delta, s)}} \leq O(i)/i = O(1)$. Таким образом, за одну итерацию объем памяти в среднем увеличится на $O(1)$, а вся структура займет $O(n)$ памяти.

Остается только получить время работы алгоритма, что довольно просто. Уже известно, что добавление отрезка занимает $O(h+k)$, но ожидаемое значение составляет $O(\log n + 1) = O(\log n)$, а для $n$ отрезков потребуется $O(n\log n)$ времени.

## Пример реализации алгоритма
Ниже приведена интерактивная реализация алгоритма для отрезков без внутренних пересечений. 
Также при клике рядом с вершиной какого-либо отрезка клик будет "наводиться" на эту вершину.

In [ ]:
%matplotlib notebook
examples.interactive_example()